### notebook for generating thresholds to subsample 10% of iucn data to pick threshold, same as v1, except we have the same amount of presence and absence data

In [75]:
import torch
import pandas as pd
import numpy as np
import json
import os
import sys
from sklearn.metrics import f1_score, precision_recall_curve
from sklearn.model_selection import train_test_split
import math


In [55]:
import datasets
import models
import utils

In [56]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [57]:
# load model
train_params = torch.load('./pretrained_models/model_an_full_input_enc_sin_cos_hard_cap_num_per_class_1000.pt', map_location='cpu')
model = models.get_model(train_params['params'])
model.load_state_dict(train_params['state_dict'], strict=True)
model = model.to(DEVICE)
model.eval()

ResidualFCNet(
  (class_emb): Linear(in_features=256, out_features=47375, bias=False)
  (feats): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=256, bias=True)
    )
    (3): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=256, bias=True)
    )
    (4): ResLayer(
      (nonlin1): ReLU(inplace=True)
      (nonlin2): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=256, out_features=256, bias=True)
      (w2): Linear(in_features=256, out_features=

In [58]:
#load reference from iucn
with open(os.path.join('./data/eval/iucn/', 'iucn_res_5.json'), 'r') as f:
            data = json.load(f)
species_ids = list((data['taxa_presence'].keys()))

In [59]:
if train_params['params']['input_enc'] in ['env', 'sin_cos_env']:
    raster = datasets.load_env()
else:
    raster = None
enc = utils.CoordEncoder(train_params['params']['input_enc'], raster=raster)

In [60]:
obs_locs = np.array(data['locs'], dtype=np.float32)
obs_locs = torch.from_numpy(obs_locs).to('cpu')
loc_feat = enc.encode(obs_locs)

In [61]:
classes_of_interest = torch.zeros(len(species_ids), dtype=torch.int64)
taxa_ids = torch.zeros(len(species_ids), dtype=torch.int64)
for tt_id, tt in enumerate(species_ids):
    class_of_interest = np.array([train_params['params']['class_to_taxa'].index(int(tt))])
    classes_of_interest[tt_id] = torch.from_numpy(class_of_interest)
    taxa_ids[tt_id] = int(tt)

In [62]:
with torch.no_grad():
    loc_emb = model(loc_feat, return_feats=True)
    wt = model.class_emb.weight[classes_of_interest, :]

In [63]:
def f1_at_thresh(y_true, y_pred, thresh, type = 'binary'):
    y_thresh = y_pred > thresh
    return f1_score(y_true, y_thresh, average=type)

In [77]:
output = list()
for tt_id, taxa in enumerate(taxa_ids):
    wt_1 = wt[tt_id,:]
    preds = torch.sigmoid(torch.matmul(loc_emb, wt_1)).cpu().numpy()
    taxa = taxa.item()
    species_locs = data['taxa_presence'].get(str(taxa))
    y_test = np.zeros(preds.shape, int)
    y_test[species_locs] = 1

    #split into 5% used for setting thresholds 95% for final eval

    # Presences
    indices_of_ones = np.where(y_test == 1)[0]
    subset_size = math.ceil((len(indices_of_ones) * 0.05))
    random_indices_ones = np.random.choice(indices_of_ones, subset_size, replace=False)

    # Absences
    indices_of_zeros = np.where(y_test == 0)[0]
    random_indices_zeros = np.random.choice(indices_of_zeros, subset_size, replace=False)

    chosen_indices_mask = np.zeros_like(y_test, dtype=bool)
    chosen_indices_mask[np.concatenate((random_indices_zeros, random_indices_ones))] = True

    thresh_y_test = y_test[chosen_indices_mask]
    thresh_preds = preds[chosen_indices_mask]
    eval_y_test = y_test[~chosen_indices_mask]
    eval_preds = preds[~chosen_indices_mask]
    
    #calculate thresholds 
    precision, recall, thresholds = precision_recall_curve(thresh_y_test, thresh_preds)
    p1 = (2 * precision * recall)
    p2 = (precision + recall)
    out = np.zeros( (len(p1)) )
    fscore = np.divide(p1,p2, out=out, where=p2!=0)
    index = np.argmax(fscore)
    thres = thresholds[index]

    #evaluate performance
    f1 = f1_at_thresh(eval_y_test, eval_preds, thres)
    
    row = {
        "taxon_id": taxa,
        "thres": thres,
        "fscore": f1,
        "#presence_absence_pairs": subset_size,
    }
    row_dict = dict(row)
    output.append(row_dict)

    if(tt_id%100==0):
            print(tt_id)

output_pd = pd.DataFrame(output)
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


In [79]:
output_pd.fscore.mean()

0.4742468490769256

In [80]:
output_pd.to_csv('./results/f1_scores/iucn_subset_scores_v2.csv')